# Демонстрация магических методов

In [1]:
import os

class ClassA():
    """Демонстрация поведения магических методов и атрибутов"""

    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls) # Возвращается созданный экземпляр класса object()
        print('Called: __new__({}, {}, {})'.format(cls.__name__, args, kwargs))
        return obj

    def __init__(self, *args, **kwargs):
        print('Called: __init__({}, {}, {})'.format(self.__class__.__name__, args, kwargs))
        self.my_attr_a = args[0]
        self.my_container = [1, 2, 3, 4, 5]
        self.my_current_iter = 0

    def __call__(self, value):
        print('Called: __call__({}, {})'.format(self.__class__.__name__, value))
        return value

    # https://stackoverflow.com/questions/22616559/use-cases-for-property-vs-descriptor-vs-getattribute
    # Магические геттеры/сеттеры атрибутов (__getattr__, __getattribute__, __setattr__, __delattr__)
    #             vs
    # Дескрипторы атрибутов (__get__, __set__, __delete__) и декоратор @property в частности
    #
    # Первый набор используется для ВСЕХ атрибутов экземпляра класса
    # Второй набор используется конкретно для какого-то атрибута или метода

    def __getattr__(self, name):
        return 'Доступ к несуществующему атрибуту "{}"'.format(name)

    def __getattribute__(self, name):
        # print('Called: __getattribute__({})'.format(name))
        return object.__getattribute__(self, name)

    def __setattr__(self, name, value):
        print('Called: __setattr__({}, {})'.format(name, value))
        return object.__setattr__(self, name, value)

    def __delattr__(self, name):
        print('Called: __delattr__({})'.format(name))
        object.__delattr__(self, name)

    def __get__(self, obj, obj_type):
        print('Called: __get__({}, {})'.format(obj.__class__.__name__, obj_type))
        return

    def __set__(self, obj, value):
        print('Called: __set__({}, {})'.format(obj.__class__.__name__, value))

    def __delete__(self, obj):
        print('Called: __delete__({})'.format(obj.__class__.__name__))

    def __getitem__(self, key):
        print('Called: __getitem__({})'.format(key))
        return self.my_container[key]

    def __setitem__(self, key, value):
        print('Called: __setitem__({}, {})'.format(key, value))
        self.my_container[key] = '*' + str(value) + '*'

    def __iter__(self):
        print('Called: __iter__({})'.format(self.__class__.__name__))
        return self

    def __next__(self):
        print('Called: __next__({})'.format(self.__class__.__name__))
        
        if self.my_current_iter >= len(self.my_container):
            raise StopIteration
        
        result = self.my_container[self.my_current_iter]
        self.my_current_iter += 1  # Увеличиваем и сохраняем текущее значение итерации
        return result
        

    def __enter__(self):
        print('Called: __enter__({})'.format(self.__class__.__name__))
        return # Возвращается в переменную которая идет после ключевого слова as. В нее можно ничего возвращать

    def __exit__(self, exc_type, exc_value, traceback):
        print('Called: __exit__({}, {}, {}, {})'.format(self.__class__.__name__, exc_type, exc_value, traceback))
        # Здесь может производится, например, закрытие файла
        # или отлов исключения внутри блока with ... as ...
        if exc_type:
            print('Произошло исключение внутри блока with')
            return True

    def __add__(self, obj):
        if isinstance(obj, ClassA):
            result = self.my_attr_a + obj.my_attr_a
        elif isinstance(obj, int):
            result = self.my_attr_a + obj

        return str(result) + ' bla-bla-bla' # Подмешиваем в результат всё что хотим

    def __str__(self):
        return '{}-{}'.format(self.__class__.__name__, str(self.my_container))

    def __repr__(self):
        return 'Экземпляр класса {}'.format(self.__class__.__name__)

    def __hash__(self):
        return hash(self.my_attr_a)

    def __eq__(self, obj):
        return self.my_attr_a == obj.my_attr_a

    def __del__(self):
        print('Called: __del__({}), Goodbye!'.format(self.__class__.__name__))

    def myMethod(self, name='Vasya Pupkin'):
        """Это док строка метода myMethod"""

###############################################################################################

class ClassB(ClassA):
    my_attr_b = ClassA('value3')

objA = ClassA('value1')
objB = ClassB('value2')
objC = ClassB('value1')


Called: __new__(ClassA, ('value3',), {})
Called: __init__(ClassA, ('value3',), {})
Called: __setattr__(my_attr_a, value3)
Called: __setattr__(my_container, [1, 2, 3, 4, 5])
Called: __setattr__(my_current_iter, 0)
Called: __new__(ClassA, ('value1',), {})
Called: __init__(ClassA, ('value1',), {})
Called: __setattr__(my_attr_a, value1)
Called: __setattr__(my_container, [1, 2, 3, 4, 5])
Called: __setattr__(my_current_iter, 0)
Called: __new__(ClassB, ('value2',), {})
Called: __init__(ClassB, ('value2',), {})
Called: __setattr__(my_attr_a, value2)
Called: __setattr__(my_container, [1, 2, 3, 4, 5])
Called: __setattr__(my_current_iter, 0)
Called: __new__(ClassB, ('value1',), {})
Called: __init__(ClassB, ('value1',), {})
Called: __setattr__(my_attr_a, value1)
Called: __setattr__(my_container, [1, 2, 3, 4, 5])
Called: __setattr__(my_current_iter, 0)


### Атрибут `__name__`  
Возвращает имя модуля, класса или функции/метода:

In [2]:
__name__

'__main__'

In [3]:
ClassA.__name__

'ClassA'

In [4]:
objA.myMethod.__name__

'myMethod'

### Атрибут `__module__`

Возвращает имя модуля экземпляра класса:

In [5]:
objA.__module__

'__main__'

### Атрибут `__class__`

Возвращает имя типа (метакласса) модуля/класса/объекта:

In [6]:
os.__class__

module

In [7]:
ClassA.__class__

type

In [8]:
objA.__class__

__main__.ClassA

### Атрибут `__bases__`

Возвращает кортеж родительских классов:

In [9]:
ClassA.__bases__

(object,)

In [10]:
ClassB.__bases__

(__main__.ClassA,)

### Метод `__subclasses__()`

Возвращает список дочерних классов:

In [11]:
ClassA.__subclasses__()

[__main__.ClassB]

In [12]:
ClassB.__subclasses__()

[]

### Атрибут `__mro__`

MRO - **M**ethod **R**esolution **O**rder

Возвращает список иерархии классов (наследование):

In [13]:
ClassA.__mro__

(__main__.ClassA, object)

In [14]:
ClassB.__mro__

(__main__.ClassB, __main__.ClassA, object)

### Атрибут `__doc__`

Возвращает документацию модуля/класса/метода:

In [15]:
ClassA.__doc__

'Демонстрация поведения магических методов и атрибутов'

In [16]:
objA.__doc__

'Демонстрация поведения магических методов и атрибутов'

In [17]:
objA.myMethod.__doc__

'Это док строка метода myMethod'

### Метод `__str__()`

Возвращает текстовое представление объекта. Также существует встроенная функция `str()`:

In [18]:
objA.__str__()

'ClassA-[1, 2, 3, 4, 5]'

In [19]:
str(objA)

'ClassA-[1, 2, 3, 4, 5]'

### Метод `__repr__()`

Возвращает текстовое представление объекта в структурах данных (списки, словари и т.д.). Также существует встроенная функция `repr()`:

In [20]:
[objA, objB, objA ]

[Экземпляр класса ClassA, Экземпляр класса ClassB, Экземпляр класса ClassA]

In [21]:
repr(objA)

'Экземпляр класса ClassA'

### Метод `__dir__()`

Возвращает список методов класса. Также существует встроенная функция `dir()`:

In [22]:
objA.__dir__()

['my_attr_a',
 'my_container',
 'my_current_iter',
 '__module__',
 '__doc__',
 '__new__',
 '__init__',
 '__call__',
 '__getattr__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__get__',
 '__set__',
 '__delete__',
 '__getitem__',
 '__setitem__',
 '__iter__',
 '__next__',
 '__enter__',
 '__exit__',
 '__add__',
 '__str__',
 '__repr__',
 '__hash__',
 '__eq__',
 '__del__',
 'myMethod',
 '__dict__',
 '__weakref__',
 '__lt__',
 '__le__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [23]:
dir(ClassA)

['__add__',
 '__call__',
 '__class__',
 '__del__',
 '__delattr__',
 '__delete__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'myMethod']

### Атрибут `__defaults__`

Возвращает кортеж дефолтных значений атрибутов функции/метода:

In [24]:
objA.myMethod.__defaults__

('Vasya Pupkin',)

### Атрибут `__slots__`

Атрибут позволяет держать ограниченный набор атрибутов объекта.

In [25]:
objA.__slots__

'Доступ к несуществующему атрибуту "__slots__"'

### Атрибут `__dict__`

Возвращает словарь атрибутов/методов и их значений класса/объекта:

In [26]:
ClassA.__dict__

mappingproxy({'__add__': <function __main__.ClassA.__add__>,
              '__call__': <function __main__.ClassA.__call__>,
              '__del__': <function __main__.ClassA.__del__>,
              '__delattr__': <function __main__.ClassA.__delattr__>,
              '__delete__': <function __main__.ClassA.__delete__>,
              '__dict__': <attribute '__dict__' of 'ClassA' objects>,
              '__doc__': 'Демонстрация поведения магических методов и атрибутов',
              '__enter__': <function __main__.ClassA.__enter__>,
              '__eq__': <function __main__.ClassA.__eq__>,
              '__exit__': <function __main__.ClassA.__exit__>,
              '__get__': <function __main__.ClassA.__get__>,
              '__getattr__': <function __main__.ClassA.__getattr__>,
              '__getattribute__': <function __main__.ClassA.__getattribute__>,
              '__getitem__': <function __main__.ClassA.__getitem__>,
              '__hash__': <function __main__.ClassA.__hash__>,

In [27]:
objA.__dict__

{'my_attr_a': 'value1', 'my_container': [1, 2, 3, 4, 5], 'my_current_iter': 0}

### Атрибут `__file__`

Возвращает путь к файлу класса в котором вызван атрибут:

In [28]:
os.__file__

'/usr/lib/python3.6/os.py'

### Метод `__hash__()`

Реализует функцию хеширования которая используется, например, когда мы получаем ключи в словаре. Также существует встроенная функция `hash()`:

In [29]:
objA.__hash__()

-3639271227474053404

In [30]:
hash(objA)

-3639271227474053404

### Метод `__eq__()`

Метод реализует сравнение объектов. Переопределяет оператор `==`

In [31]:
objA.__eq__(objA)

True

In [32]:
objA == objA

True

In [33]:
objA == objB

False

In [34]:
objA == objC

True

### Метод `__getattr__()`

Метод вызывается при доступе к несуществующему атрибуту экземпляра класса:

In [35]:
objA.notfound

'Доступ к несуществующему атрибуту "notfound"'

### Метод `__getattribute__()`

Метод вызывается при доступе к либому атрибуту экземпляра класса.

In [36]:
objA.my_attr_a

'value1'

### Метод `__setattr__()`

Метод вызывается при присваивании значения любому атрибуту экземпляра класса:

In [37]:
objA.my_attr_a = 100

Called: __setattr__(my_attr_a, 100)


### Дескриптор. Метод `__get__()`

Метод определяет поведение при доступе к объекту как к атрибуту другого класса. Здесь `my_attr_b` это экземпляр класса `ClassA`:

In [38]:
objB.my_attr_b

Called: __get__(ClassB, <class '__main__.ClassB'>)


### Дескиптор. Метод `__set__()`

Метод определяет поведение при изменении объекта как атрибута другого класса.

In [39]:
objB.my_attr_b = 200

Called: __setattr__(my_attr_b, 200)
Called: __set__(ClassB, 200)


### Дескиптор. Метод `__delete__()`

Метод определяет поведение при удалении объекта как атрибута другого класса.

In [40]:
del objB.my_attr_b

Called: __delattr__(my_attr_b)
Called: __delete__(ClassB)


### Метод `__call__()`

Метод используется при вызове экземпляра класса, а не самого класса. Используется, например, декораторами: @mydecorator(something)

In [41]:
objA(100)

Called: __call__(ClassA, 100)


100

### Метод `__add__()`

Метод реализует поведение сложения объектов. Переопределение оператора `+`:

In [42]:
objA.__add__(55)

'155 bla-bla-bla'

In [43]:
objA + 1

'101 bla-bla-bla'

### Метод `__setitem__()`

Переопределение оператора `[]`. Метод определяет поведение объекта при присваивании значения по индексу или ключу - `obj[key] = value`:

In [44]:
objA[0] = 10

Called: __setitem__(0, 10)


In [45]:
objA.my_container

['*10*', 2, 3, 4, 5]

### Метод `__getitem__()`

Переопределение оператора `[]`. Метод определяет поведение объекта при доступе к значению по индексу или ключу - `obj[key]`:

In [46]:
objA[0]

Called: __getitem__(0)


'*10*'

In [47]:
objA.my_container

['*10*', 2, 3, 4, 5]

### Методы `__iter__()` и `__next__()`

Методы опеределяют объект по которому будет проводиться итерирация и указатель на следующий элемент итерации. Также существует встроенные функции `iter()` и `next()`.

In [48]:
for i in objA:
    print('i =', i)

Called: __iter__(ClassA)
Called: __next__(ClassA)
Called: __setattr__(my_current_iter, 1)
i = *10*
Called: __next__(ClassA)
Called: __setattr__(my_current_iter, 2)
i = 2
Called: __next__(ClassA)
Called: __setattr__(my_current_iter, 3)
i = 3
Called: __next__(ClassA)
Called: __setattr__(my_current_iter, 4)
i = 4
Called: __next__(ClassA)
Called: __setattr__(my_current_iter, 5)
i = 5
Called: __next__(ClassA)


### Методы `__enter__()` и `__exit__()`

Контекстный менеджер. Методы определяют вход и выход из контекстного менеджера `with ... as ...`.

In [49]:
with objA as a:
    print('Вошли в блок контекстного менеджера')

Called: __enter__(ClassA)
Вошли в блок контекстного менеджера
Called: __exit__(ClassA, None, None, None)
